In [5]:
import pandas as pd
from scipy.optimize import minimize


In [2]:
# Replace 'data.csv' with the path to your CSV file
df_train = pd.read_csv('26a3f9eddc82d3907db7749c34f68938_train.csv')
df_val = pd.read_csv('26a3f9eddc82d3907db7749c34f68938_val.csv')



In [6]:
df_train

,Unnamed: 0,file_path,label,antenna,datetime,pred
0,0,/mnt/nas05/data01/vincenzo/ecallisto/data/GLAS...,False,GLASGOW_01,2023-06-12 17:06:00,-3.445508
1,1,/mnt/nas05/data01/vincenzo/ecallisto/data/MRO_...,False,MRO_61,2024-03-13 14:29:00,-0.489554
2,2,/mnt/nas05/data01/vincenzo/ecallisto/data/BIR_...,False,BIR_01,2022-06-15 09:04:00,-6.871867
3,3,/mnt/nas05/data01/vincenzo/ecallisto/data/ALAS...,False,ALASKA-HAARP_62,2022-04-29 02:19:00,-6.750520
4,4,/mnt/nas05/data01/vincenzo/ecallisto/data/BIR_...,False,BIR_01,2023-03-31 04:47:00,-6.865746
...,...,...,...,...,...,...
24434,24434,/mnt/nas05/data01/vincenzo/ecallisto/data/MONG...,False,MONGOLIA-UB_01,2022-12-02 01:25:00,-5.138582
24435,24435,/mnt/nas05/data01/vincenzo/ecallisto/data/MRO_...,False,MRO_59,2024-03-06 07:29:00,-5.866956
24436,24436,/mnt/nas05/data01/vincenzo/ecallisto/data/ALAS...,False,ALASKA-HAARP_62,2022-02-11 19:27:00,-4.755669
24437,24437,/mnt/nas05/data01/vincenzo/ecallisto/data/GLAS...,False,GLASGOW_01,2023-06-30 07:03:00,-5.218491


In [29]:
import torch
import torch.nn.functional as F

def temperature_scaled_logits(logits, T):
    return logits / T

def bce_loss_with_temperature(logits, targets, T):
    # Apply temperature scaling
    scaled_logits = temperature_scaled_logits(logits, T)
    # Compute binary cross-entropy loss
    return F.binary_cross_entropy_with_logits(scaled_logits, targets)

# Assuming you have saved training and validation logits and targets
# Train and validation logits
train_logits, train_targets =  torch.tensor(df_train['pred']), torch.tensor(df_train['label'], dtype=float)
val_logits, val_targets =  torch.tensor(df_val['pred']),  torch.tensor(df_val['label'], dtype=float)

# Train loss (without temperature scaling)
train_loss = F.binary_cross_entropy_with_logits(train_logits, train_targets)

# Function to minimize for T
def loss_diff(T):
    T = torch.tensor(T)
    val_loss = bce_loss_with_temperature(val_logits, val_targets, T)
    # Minimize the difference between train loss and val loss
    return (val_loss - train_loss).abs()

# Optimize T (use a method like scipy.optimize.minimize or grid search)


In [27]:
bce_loss_with_temperature(train_logits, train_targets, 5)

tensor(0.3461, dtype=torch.float64)

In [20]:
from scipy.optimize import minimize

In [31]:
minimize(loss_diff, 5)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.017933786196292795
        x: [ 1.066e+00]
      nit: 4
      jac: [ 4.992e-07]
 hess_inv: [[ 5.725e+00]]
     nfev: 20
     njev: 10

In [49]:
bce_loss_with_temperature(train_logits, train_targets,  1.066)

tensor(0.1048, dtype=torch.float64)

In [48]:
bce_loss_with_temperature(val_logits, val_targets, 1.066)

tensor(0.1221, dtype=torch.float64)